
# Read and preprocess genomic data from S3 and store features in SageMaker FeatureStore


In [9]:
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io, os
from time import gmtime, strftime, sleep
import time
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role
from sagemaker.feature_store.feature_group import FeatureGroup


## Set up SageMaker FeatureStore


In [11]:
region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

role = get_execution_role()
s3_client = boto3.client('s3', region_name=region)

default_s3_bucket_name = feature_store_session.default_bucket()
prefix = 'sagemaker-featurestore-demo'


## Get data from S3


In [12]:
# Get data from S3 
bucket_gen = 'guidance-multimodal-hcls-healthai-machinelearning/preprocessing'
#bucket_gen = <S3-bucket-name>

# Genomic data 
data_key_gen = 'final_genomic_df.csv'
#data_key_gen = <file-name.csv>

data_location_gen = 's3://{}/{}'.format(bucket_gen, data_key_gen)
data_genomic = pd.read_csv(data_location_gen)


## Preprocess Data


In [20]:
# Replacing NaN with zeros
data_genomic_1 = data_genomic.copy()
data_genomic_1 = data_genomic_1.replace(np.nan, 0)
data_genomic_1 = data_genomic_1.astype(str)

# Converting all diagnosis codes to a set
data_genomic_1[['alzheimers_prediction', 'coronary_heart_disease_prediction', 'stroke_prediction', 'hypertension_prediction']] = '0'
data_genomic_pred = data_genomic_1.copy()
for i in range(len(data_genomic_1)):
    data_genomic_pred.iloc[i]['diagnosisCode'] = set(data_genomic_pred.iloc[i]['diagnosisCode'].replace('\'', '').replace(' ', '').replace('{', '').replace('}', '').split(','))

# Adding a column for prediction of Alzheimer's disease code '26929004'
# Adding a column for prediction of Coronary heart disease '53741008'
# Adding a column for prediction of Stroke code '230690007'
# Adding a column for prediction of Hypertension code '59621000'
for i in range(len(data_genomic_pred)):
    if "26929004" in (data_genomic_pred.iloc[i]['diagnosisCode']):
        data_genomic_pred.iloc[i]['alzheimers_prediction']  =  '1'
    if "53741008" in (data_genomic_pred.iloc[i]['diagnosisCode']):
        data_genomic_pred.iloc[i]['coronary_heart_disease_prediction']  =  '1'
    if "230690007" in (data_genomic_pred.iloc[i]['diagnosisCode']):
        data_genomic_pred.iloc[i]['stroke_prediction']  =  '1'
    if "59621000" in (data_genomic_pred.iloc[i]['diagnosisCode']):
        data_genomic_pred.iloc[i]['hypertension_prediction']  =  '1'
print("Patients with Alzheimer's disease = ", len(data_genomic_pred[data_genomic_pred['alzheimers_prediction']=='1']))
print("Patients with Coronary Heart disease = ", len(data_genomic_pred[data_genomic_pred['coronary_heart_disease_prediction']=='1']))
print("Patients with Stroke = ", len(data_genomic_pred[data_genomic_pred['stroke_prediction']=='1']))
print("Patients with Hypertension = ", len(data_genomic_pred[data_genomic_pred['hypertension_prediction']=='1']))

# Delete columns with leakage and features irrelevant for model training
list_delete_cols = ['diagnosisDescription', 'diagnosisCode']
data_genomic_pred.drop(list_delete_cols, axis=1, inplace=True)

data_genomic_pred.head(10)

Patients with Alzheimer's disease =  19
Patients with Coronary Heart disease =  26
Patients with Stroke =  72
Patients with Hypertension =  47


,patientID,gene_info,clinical_significance,contigname,start,referenceallele,alternatealleles,phased,calls,alzheimers_prediction,coronary_heart_disease_prediction,stroke_prediction,hypertension_prediction
0,0074596f-5fd0-7965-db0f-cce71c81567d,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 31667848, 12351625, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",0,0,0,0
1,0618424e-ed51-3100-ea5c-e46492bfd65b,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 12351625, 31667848, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",0,0,0,0
2,06cc033a-f09a-0fb2-4a1a-4c4d99d88839,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 31667848, 12351625, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",1,0,1,1
3,06d43bcb-8322-3c2b-40f1-189b2852b5ab,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 12351625, 31667848, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",0,0,0,0
4,08acefb1-271e-01d2-1bb6-aae9f65dbb42,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 12351625, 31667848, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",0,0,1,0
5,0b73e9c9-18c0-5c0e-c5d5-95d7ca56c050,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 31667848, 12351625, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",1,0,1,1
6,0bb4b1f7-4128-89ef-a023-b81379c30f4f,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 31667848, 12351625, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",0,0,0,0
7,0c8ad7c0-b403-98d1-d6c9-ff1dbbce7d7c,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 31667848, 12351625, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",0,0,1,1
8,0f1c1042-f103-e85b-1a7a-03442861947d,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 12351625, 31667848, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",0,0,0,0
9,14e884a7-dc93-7909-9585-2857c6eb29e2,"{'SOD3', None, 'TNF', 'LPL', 'F5', 'PON1', 'PP...",{'Conflicting_interpretations_of_pathogenicity...,"{'14', '9', '6', '16', '17', '8', '10', '11', ...","{53767041, 44919688, 12351625, 31667848, 44908...","{'T', 'G', 'A', 'C'}","{'[A]', '[C]', '[G]', '[T]'}",{False},"{'[1, 1]', '[0, 1]'}",0,0,1,1



## Ingest data into FeatureStore


In [18]:
genomic_feature_group_name = 'genomic-feature-group'
genomic_feature_group = FeatureGroup(name=genomic_feature_group_name, sagemaker_session=feature_store_session)

current_time_sec = int(round(time.time()))

def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        print (label)
        if data_frame.dtypes[label] == 'object':
            data_frame[label] = data_frame[label].astype("str").astype("string")

# Cast object dtype to string. SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(data_genomic_pred)

# Record identifier and event time feature names
record_identifier_feature_name = "patientID"
event_time_feature_name = "EventTime"

# Append EventTime feature
data_genomic_pred[event_time_feature_name] = pd.Series([current_time_sec]*len(data_genomic_pred), dtype="float64")

## If event time generates NaN
data_genomic_pred[event_time_feature_name] = data_genomic_pred[event_time_feature_name].fillna(0)

# Load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
genomic_feature_group.load_feature_definitions(data_frame=data_genomic_pred); # output is suppressed


def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(15)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")

genomic_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True
)

wait_for_feature_group_creation_complete(feature_group=genomic_feature_group)

genomic_feature_group.ingest(
    data_frame=data_genomic_pred, max_workers=5, wait=True
)

patientID
gene_info
clinical_significance
contigname
start
referenceallele
alternatealleles
phased
calls
alzheimers_prediction
coronary_heart_disease_prediction
stroke_prediction
hypertension_prediction
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup genomic-feature-group successfully created.


IngestionManagerPandas(feature_group_name='genomic-feature-group', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fbbb3b59fd0>, sagemaker_session=<sagemaker.session.Session object at 0x7fbbb6128110>, max_workers=5, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7fbbc42058d0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])